In [1]:
# Importar librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month
import os

In [2]:
spark = SparkSession.builder \
    .appName("SECOP_Ingesta") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 01:51:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(f"Spark Version: {spark.version}")
print(f"Spark Master: {spark.sparkContext.master}")

Spark Version: 3.5.0
Spark Master: local[*]


In [4]:
import requests
import json


In [5]:
api_url = "https://www.datos.gov.co/resource/jbjy-vk9h.json?$query=SELECT%0A%20%20%60nombre_entidad%60%2C%0A%20%20%60nit_entidad%60%2C%0A%20%20%60departamento%60%2C%0A%20%20%60ciudad%60%2C%0A%20%20%60localizaci_n%60%2C%0A%20%20%60orden%60%2C%0A%20%20%60sector%60%2C%0A%20%20%60rama%60%2C%0A%20%20%60entidad_centralizada%60%2C%0A%20%20%60proceso_de_compra%60%2C%0A%20%20%60id_contrato%60%2C%0A%20%20%60referencia_del_contrato%60%2C%0A%20%20%60estado_contrato%60%2C%0A%20%20%60codigo_de_categoria_principal%60%2C%0A%20%20%60descripcion_del_proceso%60%2C%0A%20%20%60tipo_de_contrato%60%2C%0A%20%20%60modalidad_de_contratacion%60%2C%0A%20%20%60justificacion_modalidad_de%60%2C%0A%20%20%60fecha_de_firma%60%2C%0A%20%20%60fecha_de_inicio_del_contrato%60%2C%0A%20%20%60fecha_de_fin_del_contrato%60%2C%0A%20%20%60condiciones_de_entrega%60%2C%0A%20%20%60tipodocproveedor%60%2C%0A%20%20%60documento_proveedor%60%2C%0A%20%20%60proveedor_adjudicado%60%2C%0A%20%20%60es_grupo%60%2C%0A%20%20%60es_pyme%60%2C%0A%20%20%60habilita_pago_adelantado%60%2C%0A%20%20%60liquidaci_n%60%2C%0A%20%20%60obligaci_n_ambiental%60%2C%0A%20%20%60obligaciones_postconsumo%60%2C%0A%20%20%60reversion%60%2C%0A%20%20%60origen_de_los_recursos%60%2C%0A%20%20%60destino_gasto%60%2C%0A%20%20%60valor_del_contrato%60%2C%0A%20%20%60valor_de_pago_adelantado%60%2C%0A%20%20%60valor_facturado%60%2C%0A%20%20%60valor_pendiente_de_pago%60%2C%0A%20%20%60valor_pagado%60%2C%0A%20%20%60valor_amortizado%60%2C%0A%20%20%60valor_pendiente_de%60%2C%0A%20%20%60valor_pendiente_de_ejecucion%60%2C%0A%20%20%60estado_bpin%60%2C%0A%20%20%60c_digo_bpin%60%2C%0A%20%20%60anno_bpin%60%2C%0A%20%20%60saldo_cdp%60%2C%0A%20%20%60saldo_vigencia%60%2C%0A%20%20%60espostconflicto%60%2C%0A%20%20%60dias_adicionados%60%2C%0A%20%20%60puntos_del_acuerdo%60%2C%0A%20%20%60pilares_del_acuerdo%60%2C%0A%20%20%60urlproceso%60%2C%0A%20%20%60nombre_representante_legal%60%2C%0A%20%20%60nacionalidad_representante_legal%60%2C%0A%20%20%60domicilio_representante_legal%60%2C%0A%20%20%60tipo_de_identificaci_n_representante_legal%60%2C%0A%20%20%60identificaci_n_representante_legal%60%2C%0A%20%20%60g_nero_representante_legal%60%2C%0A%20%20%60presupuesto_general_de_la_nacion_pgn%60%2C%0A%20%20%60sistema_general_de_participaciones%60%2C%0A%20%20%60sistema_general_de_regal_as%60%2C%0A%20%20%60recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_%60%2C%0A%20%20%60recursos_de_credito%60%2C%0A%20%20%60recursos_propios%60%2C%0A%20%20%60ultima_actualizacion%60%2C%0A%20%20%60codigo_entidad%60%2C%0A%20%20%60codigo_proveedor%60%2C%0A%20%20%60fecha_inicio_liquidacion%60%2C%0A%20%20%60fecha_fin_liquidacion%60%2C%0A%20%20%60objeto_del_contrato%60%2C%0A%20%20%60duraci_n_del_contrato%60%2C%0A%20%20%60nombre_del_banco%60%2C%0A%20%20%60tipo_de_cuenta%60%2C%0A%20%20%60n_mero_de_cuenta%60%2C%0A%20%20%60el_contrato_puede_ser_prorrogado%60%2C%0A%20%20%60fecha_de_notificaci_n_de_prorrogaci_n%60%2C%0A%20%20%60nombre_ordenador_del_gasto%60%2C%0A%20%20%60tipo_de_documento_ordenador_del_gasto%60%2C%0A%20%20%60n_mero_de_documento_ordenador_del_gasto%60%2C%0A%20%20%60nombre_supervisor%60%2C%0A%20%20%60tipo_de_documento_supervisor%60%2C%0A%20%20%60n_mero_de_documento_supervisor%60%2C%0A%20%20%60nombre_ordenador_de_pago%60%2C%0A%20%20%60tipo_de_documento_ordenador_de_pago%60%2C%0A%20%20%60n_mero_de_documento_ordenador_de_pago%60%2C%0A%20%20%60documentos_tipo%60%2C%0A%20%20%60descripcion_documentos_tipo%60%0AWHERE%0A%20%20caseless_one_of(%60departamento%60%2C%20%22Distrito%20Capital%20de%20Bogot%C3%A1%22)%0A%20%20AND%20%60fecha_de_firma%60%0A%20%20%20%20%20%20%20%20BETWEEN%20%222025-10-01T19%3A14%3A39%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20%20%20%20AND%20%222025-12-31T23%3A59%3A00%22%20%3A%3A%20floating_timestamp"

In [6]:
response = requests.get(api_url)
data = response.json()

In [7]:
pip install sodapy


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sodapy import Socrata

client = Socrata("www.datos.gov.co", None)

results = client.get(
    "jbjy-vk9h",
    query="""
    SELECT *
    WHERE 
        departamento = "Distrito Capital de Bogotá"
    AND
        fecha_de_firma >= '2025-01-01T00:00:00'
    AND
        fecha_de_firma < '2025-04-01T00:00:00'
    LIMIT 150000
    """
)


26/02/02 01:51:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
len(results)

132642

In [10]:

import pandas as pd
import os

DATA_PATH = "/app/data"

df = pd.DataFrame.from_records(results)

df.to_csv(
    os.path.join(DATA_PATH, "SECOP_II_Contratos_Electronicos_Q1_2025.csv"),
    index=False,
    encoding="utf-8"
)

print("CSV guardado correctamente en /app/data")



CSV guardado correctamente en /app/data


In [11]:
df_spark = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/app/data/SECOP_II_Contratos_Electronicos_Q1_2025.csv")
)


In [12]:
len(df_spark.columns)


87

In [13]:
df_spark.printSchema()


root
 |-- nombre_entidad: string (nullable = true)
 |-- nit_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- localizaci_n: string (nullable = true)
 |-- orden: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- rama: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- proceso_de_compra: string (nullable = true)
 |-- id_contrato: string (nullable = true)
 |-- referencia_del_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- tipo_de_contrato: string (nullable = true)
 |-- modalidad_de_contratacion: string (nullable = true)
 |-- justificacion_modalidad_de: string (nullable = true)
 |-- fecha_de_firma: string (nullable = true)
 |-- fecha_de_inicio_del_contrato: string (nullable = true)
 |-- fecha_de_fin_del_contrato: string

In [14]:
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import DoubleType, LongType, IntegerType


In [15]:
fecha_cols = [
    "fecha_de_firma",
    "fecha_de_inicio_del_contrato",
    "fecha_de_fin_del_contrato",
    "ultima_actualizacion",
    "fecha_inicio_liquidacion",
    "fecha_fin_liquidacion",
    "fecha_de_notificaci_n_de_prorrogaci_n"
]

for c in fecha_cols:
    df_spark = df_spark.withColumn(
        c,
        to_timestamp(col(c))
    )


In [16]:
double_cols = [
    "valor_del_contrato",
    "valor_de_pago_adelantado",
    "valor_facturado",
    "valor_pendiente_de_pago",
    "valor_pagado",
    "valor_amortizado",
    "valor_pendiente_de",
    "valor_pendiente_de_ejecucion",
    "presupuesto_general_de_la_nacion_pgn",
    "sistema_general_de_participaciones",
    "recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_",
    "recursos_de_credito",
    "recursos_propios"
]

for c in double_cols:
    df_spark = df_spark.withColumn(
        c,
        col(c).cast(DoubleType())
    )


In [17]:
int_cols = [
    "dias_adicionados",
    "codigo_entidad",
    "codigo_proveedor",
    "sistema_general_de_regal_as"
]

for c in int_cols:
    df_spark = df_spark.withColumn(
        c,
        col(c).cast(IntegerType())
    )


In [18]:
long_cols = [
    "saldo_cdp",
    "saldo_vigencia"
]

for c in long_cols:
    df_spark = df_spark.withColumn(
        c,
        col(c).cast(LongType())
    )


In [19]:
df_spark.printSchema()


root
 |-- nombre_entidad: string (nullable = true)
 |-- nit_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- localizaci_n: string (nullable = true)
 |-- orden: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- rama: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- proceso_de_compra: string (nullable = true)
 |-- id_contrato: string (nullable = true)
 |-- referencia_del_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- tipo_de_contrato: string (nullable = true)
 |-- modalidad_de_contratacion: string (nullable = true)
 |-- justificacion_modalidad_de: string (nullable = true)
 |-- fecha_de_firma: timestamp (nullable = true)
 |-- fecha_de_inicio_del_contrato: timestamp (nullable = true)
 |-- fecha_de_fin_del_contrato: 

In [20]:
import os

os.getcwd()


'/app/notebooks'

In [21]:
os.listdir("/")


['bin',
 'etc',
 'sbin',
 'var',
 'media',
 'lib',
 'boot',
 'opt',
 'root',
 'mnt',
 'lib64',
 'usr',
 'sys',
 'tmp',
 'srv',
 'run',
 'dev',
 'proc',
 'home',
 'app',
 '.dockerenv']

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("SECOP EDA") \
    .master("local[*]") \
    .getOrCreate()

bronze_path = "/app/data/lakehouse/bronze/secop"
df = spark.read.parquet(bronze_path)

print("Registros:", df.count())
df.printSchema()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 02:23:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/02 02:23:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Registros: 283201
root
 |-- nombre_entidad: string (nullable = true)
 |-- nit_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- localizaci_n: string (nullable = true)
 |-- orden: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- rama: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- proceso_de_compra: string (nullable = true)
 |-- id_contrato: string (nullable = true)
 |-- referencia_del_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- tipo_de_contrato: string (nullable = true)
 |-- modalidad_de_contratacion: string (nullable = true)
 |-- justificacion_modalidad_de: string (nullable = true)
 |-- fecha_de_firma: string (nullable = true)
 |-- fecha_de_inicio_del_contrato: string (nullable = true)
 |-- fecha_de_fin_de

In [5]:
from pyspark.sql.functions import col

df_eda = (
    df
    .withColumn("valor_del_contrato", col("valor_del_contrato").cast("double"))
)



In [6]:
df_eda.select("valor_del_contrato").describe().show()


[Stage 9:============================================>              (6 + 2) / 8]

+-------+--------------------+
|summary|  valor_del_contrato|
+-------+--------------------+
|  count|              132642|
|   mean| 9.373301111717254E7|
| stddev|1.9816579423350778E9|
|    min|                 0.0|
|    max|    4.08987300811E11|
+-------+--------------------+



In [8]:
df_eda \
  .orderBy(col("valor_del_contrato").desc()) \
  .select("nombre_entidad", "departamento", "valor_del_contrato") \
  .show(10, truncate=False)


[Stage 15:===========================================>              (6 + 2) / 8]

+------------------------------------------------------------------+--------------------------+------------------+
|nombre_entidad                                                    |departamento              |valor_del_contrato|
+------------------------------------------------------------------+--------------------------+------------------+
|DIRECCIÓN GENERAL DE SANIDAD MILITAR+                             |Distrito Capital de Bogotá|4.08987300811E11  |
|AGENCIA LOGISTICA DE LAS FUERZAS MILITARES                        |Distrito Capital de Bogotá|2.25E11           |
|MINISTERIO DE EDUCACION NACIONAL (MEN)                            |Distrito Capital de Bogotá|1.98299778474E11  |
|RNEC                                                              |Distrito Capital de Bogotá|1.50586014037E11  |
|ICBF SEDE NACIONAL                                                |Distrito Capital de Bogotá|1.4549721374E11   |
|(Secretaría Distrital de Integración Social)                      |Distrito Cap

Realizamos un análisis exploratorio de los datos de contratación pública (SECOP) con el fin de identificar patrones generales, valores atípicos y problemas de calidad de datos.
Se evidenció alta dispersión en los valores contractuales, concentración de contratos en ciertos departamentos y presencia de valores nulos, lo cual justifica la implementación de una arquitectura Lakehouse con capas Bronze, Silver y Gold.

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, min, max, stddev

spark = SparkSession.builder \
    .appName("SECOP - Estadística Exploratoria Final") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

silver_path = "/app/data/lakehouse/silver/secop"
df = spark.read.parquet(silver_path)

print(" Total de registros:", df.count())
df.printSchema()


 Total de registros: 110621
root
 |-- nombre_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- valor_del_contrato: string (nullable = true)
 |-- fecha_de_firma: string (nullable = true)
 |-- anio_firma: integer (nullable = true)
 |-- trimestre_firma: integer (nullable = true)



In [11]:
df.show(5, truncate=False)


+-------------------------------------------------------------+--------------------------+-----------+------------------+-----------------------+----------+---------------+
|nombre_entidad                                               |departamento              |ciudad     |valor_del_contrato|fecha_de_firma         |anio_firma|trimestre_firma|
+-------------------------------------------------------------+--------------------------+-----------+------------------+-----------------------+----------+---------------+
|MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIBLE               |Distrito Capital de Bogotá|Bogotá     |131654600         |2025-01-28T00:00:00.000|2025      |1              |
|IDRD - ENTIDAD OFICIAL.                                      |Distrito Capital de Bogotá|Bogotá     |25687000          |2025-03-25T00:00:00.000|2025      |1              |
|UNP                                                          |Distrito Capital de Bogotá|Bogotá     |47333333          |2025-03-20T00:

In [12]:
df.select("valor_del_contrato").describe().show()


+-------+--------------------+
|summary|  valor_del_contrato|
+-------+--------------------+
|  count|              110343|
|   mean|1.1267533110939525E8|
| stddev| 2.172195738536503E9|
|    min|                   1|
|    max|             9999999|
+-------+--------------------+



In [13]:
df.select(
    avg("valor_del_contrato").alias("promedio"),
    stddev("valor_del_contrato").alias("desviacion_estandar"),
    min("valor_del_contrato").alias("minimo"),
    max("valor_del_contrato").alias("maximo")
).show(truncate=False)


+--------------------+-------------------+------+-------+
|promedio            |desviacion_estandar|minimo|maximo |
+--------------------+-------------------+------+-------+
|1.1267533110939525E8|2.172195738536503E9|1     |9999999|
+--------------------+-------------------+------+-------+



In [16]:
df.groupBy("anio_firma") \
  .agg(count("*").alias("total_contratos")) \
  .orderBy("anio_firma") \
  .show()


+----------+---------------+
|anio_firma|total_contratos|
+----------+---------------+
|      NULL|           9259|
|      2019|              1|
|      2025|         101216|
|      2026|            132|
|      2027|              5|
|      2028|              5|
|      2030|              2|
|      2055|              1|
+----------+---------------+



In [17]:
df.groupBy("anio_firma", "trimestre_firma") \
  .agg(count("*").alias("total_contratos")) \
  .orderBy("anio_firma", "trimestre_firma") \
  .show()


+----------+---------------+---------------+
|anio_firma|trimestre_firma|total_contratos|
+----------+---------------+---------------+
|      NULL|           NULL|           9259|
|      2019|              1|              1|
|      2025|              1|         101093|
|      2025|              2|             14|
|      2025|              3|             24|
|      2025|              4|             85|
|      2026|              1|            108|
|      2026|              2|             16|
|      2026|              3|              7|
|      2026|              4|              1|
|      2027|              1|              3|
|      2027|              2|              1|
|      2027|              4|              1|
|      2028|              1|              2|
|      2028|              2|              3|
|      2030|              1|              2|
|      2055|              1|              1|
+----------+---------------+---------------+



In [18]:
df.orderBy(col("valor_del_contrato").desc()) \
  .select(
      "nombre_entidad",
      "departamento",
      "ciudad",
      "valor_del_contrato"
  ) \
  .show(10, truncate=False)


+-------------------------------------------------------------+--------------------------+-----------+------------------+
|nombre_entidad                                               |departamento              |ciudad     |valor_del_contrato|
+-------------------------------------------------------------+--------------------------+-----------+------------------+
|CAMARA DE REPRESENTANTES                                     |Distrito Capital de Bogotá|No Definido|9999999           |
|CAMARA DE REPRESENTANTES                                     |Distrito Capital de Bogotá|No Definido|9999999           |
|CAMARA DE REPRESENTANTES                                     |Distrito Capital de Bogotá|No Definido|9999999           |
|UPRA                                                         |Distrito Capital de Bogotá|Bogotá     |9999570           |
|SECRETARIA DISTRITAL DE HACIENDA                             |Distrito Capital de Bogotá|No Definido|99990000          |
|DIRECCION DE SANIDAD EJ